# 다이나믹스를 반영한 메트릭 분석: 김미령

In [1]:
from matplotlib import font_manager
from matplotlib import rc
import pickle
import re

f_path = '/Library/Fonts/Arial Unicode.ttf'
font_manager.FontProperties(fname=f_path).get_name()

rc('font', family='Arial Unicode MS')

In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from networkx.algorithms import bipartite, community
from networkx.algorithms.community import modularity, louvain_communities
import matplotlib.colors as mcolors
from collections import Counter
import pandas as pd


# 1. 로딩

In [3]:
# with open('../local_data/dynamics_attached_sheet.pkl', 'rb') as file:
with open('../local_data/dynamics_attached_sheet_20241102.pkl', 'rb') as file:
    result_based_on_today = pickle.load(file)
    result_based_on_yesterday = pickle.load(file)

In [6]:
testers = list(result_based_on_today.keys())

## 엑셀 시트로 정리

In [5]:
# def save_dict_to_excel(data_dict, excel_file_path):
#     # ExcelWriter 객체 생성
#     with pd.ExcelWriter(excel_file_path, engine=None) as writer:
#         # 딕셔너리의 각 항목에 대해 반복
#         for sheet_name, df in data_dict.items():
#             # 데이터프레임을 엑셀 시트로 저장
#             df.to_excel(writer, sheet_name=sheet_name, index=False)

In [6]:
# save_dict_to_excel(result_based_on_today, '../local_data/dynamics-and-category-assigned-results.xlsx')

In [ ]:
tester = testers[0]
tester

In [ ]:
print(testers)

In [9]:
def make_daily_mean_adjusted_df(source_df:pd.DataFrame, tester):
    '''
    trial_idx 별 평균을 구한 후, 이를 뺀 값으로 보정한 메트릭을 추가함
    *_adjusted로 추가함
    trial_idx > 0 인 경우에만 계산함
    '''
    df = source_df[tester].query('trial_idx > 0').copy()
    df['trial_pos'] = df.groupby('trial_idx').cumcount()

    # Calculate the mean for each trial_idx for precision, freshness, and satisfaction
    trial_idx_means = df.groupby('trial_idx')[['precision', 'freshness', 'satisfaction']].mean()

    # Merge the mean values back to the original dataset
    df = df.merge(trial_idx_means, on='trial_idx', suffixes=('', '_mean'))

    # Calculate the adjusted values by subtracting the mean values from the original values
    df['precision_adjusted'] = df['precision'] - df['precision_mean']
    df['freshness_adjusted'] = df['freshness'] - df['freshness_mean']
    df['satisfaction_adjusted'] = df['satisfaction'] - df['satisfaction_mean']

    return df

In [11]:
df = make_daily_mean_adjusted_df(result_based_on_today, tester)
disappearing_df = make_daily_mean_adjusted_df(result_based_on_yesterday, tester)

In [12]:
metric_cols = ['precision', 'freshness', 'satisfaction']
metric_cols_adjusted = [col + '_adjusted' for col in metric_cols]

# 2. 분석

## 2.1 기본 분석

In [12]:
# Step 1: Calculate global averages for precision, freshness, satisfaction
global_averages = df[metric_cols].mean()
# daily_averages = df[['precision', 'freshness', 'satisfaction']].mean()

# Step 2: Calculate averages per trial_idx and trial_pos
trial_idx_averages = df.groupby('trial_idx')[metric_cols].mean()
trial_pos_averages = df.groupby('trial_pos')[metric_cols].mean()

# Compare these with the global averages
trial_idx_differences = trial_idx_averages - global_averages
trial_pos_differences = trial_pos_averages - global_averages

# import ace_tools as tools; tools.display_dataframe_to_user(name="Trial Index Differences", dataframe=trial_idx_differences)
# tools.display_dataframe_to_user(name="Trial Position Differences", dataframe=trial_pos_differences)

# trial_idx_differences, trial_pos_differences


`trial_idx` 기준으로 각 메트릭과 전역 평균값과의 차이 플로팅

- 날짜가 지날 수록 감소함
- 시행 초기에 메트릭이 높으나 점차 감소하여 마이너스가 됨

In [ ]:
trial_idx_differences.plot()

`trial_pos` 기준으로 각 메트릭과 전역 평균값과의 차이 플로팅

- 확연한 차이는 아니지만 상위 위치에서 메트릭이 양수가 많음

In [ ]:
trial_pos_differences.plot()

## 2.1 (*) 일평균을 뺀 것으로 보정하여 분석 진행

In [15]:
# Step 1: Calculate global averages for precision, freshness, satisfaction
global_averages = df[metric_cols_adjusted].mean()
# daily_averages = df[['precision', 'freshness', 'satisfaction']].mean()

# Step 2: Calculate averages per trial_idx and trial_pos
trial_idx_averages = df.groupby('trial_idx')[metric_cols_adjusted].mean()
trial_pos_averages = df.groupby('trial_pos')[metric_cols_adjusted].mean()

# Compare these with the global averages
trial_idx_differences = trial_idx_averages
trial_pos_differences = trial_pos_averages

# import ace_tools as tools; tools.display_dataframe_to_user(name="Trial Index Differences", dataframe=trial_idx_differences)
# tools.display_dataframe_to_user(name="Trial Position Differences", dataframe=trial_pos_differences)

# trial_idx_differences, trial_pos_differences


In [ ]:
trial_idx_differences.plot()

In [ ]:
trial_pos_differences.plot()

## 2.2 노드 타입에 따른 메트릭 차이 분석

In [18]:
# Filter out rows with NaN values in 'type' to focus on rows where type is defined
filtered_data = df.dropna(subset=['type'])

# Step 1: Calculate averages per node type
type_averages = \
    pd.concat((filtered_data.groupby('type')[metric_cols_adjusted].mean(),
        disappearing_df.groupby('type')[metric_cols_adjusted].mean()), axis=0)

# relative_type_averages = type_averages


절대값 플로팅과 상대 차이 플로팅

- 노드 타입에 따라 precision, freshness, satisfaction에 차이 확인.
- `merging`과 `shrinking` 타입에서 상대적으로 높은 satisfaction 점수가 보임
- 반대로 `persisting` 타입에서는 비교적 낮은 점수가 나타남


In [ ]:
type_averages.plot.bar()

In [ ]:
print(' '.join([x[:2].upper() for x in type_averages.sort_values('satisfaction_adjusted', ascending=False).index]))

## 2.3 persisting 노드의 유지 기간에 따른 메트릭 분석

In [21]:
# Filter for 'persisting' type nodes and those that have a non-null streak_count
persisting_data = filtered_data[filtered_data['type'] == 'persisting'].dropna(subset=['streak_count'])

# Step 1: Calculate averages based on streak_count
streak_count_averages = persisting_data.groupby('streak_count')[metric_cols_adjusted].mean()

# streak_count_averages

# relative_streak_count_averages = streak_count_averages - global_averages


플로팅을 통한 확인

- 유지 기간이 길수록 메트릭 감소가 확연함

In [ ]:
streak_count_averages.plot.bar()

## 2.4 주요 결과와 의미

1. 평가 지표(precision, freshness, satisfaction)에 대한 글로벌 평균 대비 분석
- `trial_idx`와 평가 값의 관계:
    - `trial_idx` 0번에서 글로벌 평균보다 높은 precision, freshness, satisfaction 점수를 기록
    - 이는 첫 날 노출된 콘텐츠들이 전반적으로 사용자들로부터 긍정적인 평가를 받았음을 시사
    - `trial_idx` 가 증가함에 따라 평가 점수는 점차 글로벌 평균에 근접하거나 감소하는 경향을 보임
- `trial_pos`와 평가 값의 관계:
    - `trial_pos` 상위(0~2번)에 위치한 콘텐츠들이 전반적으로 글로벌 평균보다 높은 평가 점수를 받음
    - 이는 사용자가 처음 노출되는 콘텐츠에 더 높은 만족도를 느끼는 경향이 있음에 대한 가능성
2. 노드 타입에 따른 평가 지표의 차이
- 노드 타입별 차이:
    - `merging`과 `shrinking` 노드 타입은 다른 타입에 비해 상대적으로 높은 satisfaction 점수를 기록
    - 이는 이러한 노드들이 사용자들에게 더 긍정적인 경험을 제공했음을 시사
    - 반면, `persisting`` 노드는 전체적으로 낮은 평가 점수를 기록
    - 이는 동일한 콘텐츠가 반복적으로 노출되는 것에 대해 사용자가 피로감을 느끼거나 흥미를 잃었을 가능성을 시사
3. `persisting` 노드의 유지 기간에 따른 변화
- 유지 기간과 평가 값의 관계:
  - `persisting` 노드의 유지 기간(streak_count)이 길어질수록 precision, freshness, satisfaction 점수가 전반적으로 감소하는 경향을 보임
  - 특히, 4일 이상 유지된 경우 평가 값이 급격히 감소했습니다. 이는 사용자가 동일한 콘텐츠가 지속적으로 노출되는 것에 대한 피로감이 누적 되었을 수도...

# 3. 추가 조사

## 3.1 상관 관계

freshness와 satisfaction 사이의 상관 관계: 0.966056 으로 매우 높음

In [ ]:
df[metric_cols_adjusted].corr()


## 3.2 콘텐츠 유형과 평가 지표의 관계


1. 높은 평가를 받은 카테고리:
- 1.4.1. 국방 정책: precision, freshness, satisfaction 모두에서 높은 점수
- 5.9. 바이오테크놀로지: 모든 지표에서 높은 점수를 기록
- 2.3. 부동산 및 2.2. 증권/주식: 경제 관련 카테고리에서도 전반적으로 높은 평가를 받았습니다. 이는 경제 정보에 대한 사용자 관심이 높음을 의미

2. 낮은 평가를 받은 카테고리:
- 10.2. 기상이슈, 8.1. 국내 야구: 이들 카테고리는 모든 평가 지표에서 낮은 점수, 이 주제에 대한 관심도가 낮을 수 있음을 시사
- 7.3. 음악,  7.4. 스타/연예인: 엔터테인먼트 관련 카테고리에서도 상대적으로 낮은 평가

In [24]:
# Calculate the averages of precision, freshness, and satisfaction based on the category
category_averages = df.groupby('category')[metric_cols_adjusted].mean()

# relative_category_averages = category_averages - global_averages


In [ ]:
category_averages.sort_values(by=metric_cols_adjusted, ascending=False).head(5)

In [ ]:
category_averages.sort_values(by=metric_cols_adjusted, ascending=False).tail(5)